In [1]:
import pandas as pd
data=pd.read_excel('/content/drive/MyDrive/ai_recruitment_dataset/copy_of_prediction_data.xlsx')
data

,Unnamed: 0.1,Unnamed: 0,ID,Name,Role,Transcript,Resume,Reason for decision,Job Description,num_words_in_transcript
0,514,537,rivash0038,lahar singh,software engineer,**lahar singh: software engineer candidate int...,**lahar singh**\n**software engineer candidate...,"expected_experience : 9+ years, domains: e-com...","communicated ideas clearly and effectively., h...",956
1,214,225,benjry660,benjamin ryan,data engineer,interview transcript: data engineer position\n...,here's a sample resume for benjamin ryan apply...,cultural fit,we are looking for a skilled data engineer wit...,551
2,1408,1467,rivash0968,amisha bedi,data scientist,"**interview transcript: amisha bedi, data scie...",**candidate profile: amisha bedi**\n\n**role:*...,"expected_experience : 6-8 years, domains: heal...","lacked key technical skills for the role., nee...",612
3,1071,1122,rivash0623,kairav mishra,product manager,**interview transcript: product manager positi...,**kairav mishra: product manager**\n\nas a sea...,"expected_experience : 6-8 years, domains: tech...","had impressive experience and qualifications.,...",793
4,390,410,bradgr792,bradley gross,product manager,product manager interview transcript\n\ninterv...,here's a sample resume for bradley gross apply...,cultural fit,we are looking for a skilled product manager w...,665
...,...,...,...,...,...,...,...,...,...,...
95,1380,1438,rivash0939,charvi verma,software engineer,**interview transcript**\n\n**interviewer:** r...,**charvi verma**\n**software engineer candidat...,"expected_experience : 3-5 years, domains: e-co...","had impressive experience and qualifications.,...",961
96,546,572,rivash0073,mythily chawla,data scientist,**interview transcript**\n\n**mythily chawla:*...,**candidate profile: mythily chawla**\n\n**int...,"expected_experience : 6-8 years, domains: heal...","had insufficient experience for the position.,...",719
97,961,1008,rivash0509,ehan naidu,software engineer,**interview transcript: software engineer posi...,**candidate profile: ehan naidu**\n\n**positio...,"expected_experience : 6-8 years, domains: e-co...","lacked key technical skills for the role., str...",577
98,568,595,rivash0096,saanjh chopra,software engineer,**interview transcript: saanjh chopra - softwa...,**candidate profile: saanjh chopra**\n\n**inte...,"expected_experience : 6-8 years, domains: e-co...",demonstrated limited understanding of job requ...,827


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer()
resume_jd_similarity = []
for i in range(len(data)):
    resume = data['Resume'][i]
    jd = data['Job Description'][i]
    similarity = cosine_similarity(vectorizer.fit_transform([resume, jd]))[0, 1]
    resume_jd_similarity.append(similarity)
data['resume_jd_similarity'] = resume_jd_similarity

In [5]:
resume_transcript_similarity = []
for i in range(len(data)):
    resume = data['Resume'][i]
    transcript = data['Transcript'][i]
    similarity = cosine_similarity(vectorizer.fit_transform([resume, transcript]))[0, 1]
    resume_transcript_similarity.append(similarity)
data['resume_transcript_similarity'] = resume_transcript_similarity

In [6]:
!pip install vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.5 MB/s eta 0:00:00


In [7]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
data['sentiment'] = data['Transcript'].apply(lambda transcript: sia.polarity_scores(transcript)['compound'])
average_sentiment = data['sentiment'].mean()

In [8]:
def lexical_diversity(text):
    words = text.split()
    return len(set(words)) / len(words)

# Compute lexical diversity for each transcript
data['lexical_diversity'] = data['Transcript'].apply(lexical_diversity)

In [9]:
data['transcript_length_words'] = data['Transcript'].apply(lambda x: len(x.split()))

In [10]:
data['reason_length'] = data['Reason for decision'].str.split().apply(len)

In [11]:
data['resume_length'] = data['Resume'].apply(lambda x: len(x.split()))

In [12]:
data['word_count_ratio'] = data['transcript_length_words'] / data['resume_length']

In [13]:
def text_similarity(text1, text2):
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity

    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([text1, text2])
    return cosine_similarity(vectors[0], vectors[1])[0, 0]

data['role_transcript_similarity'] = data.apply(
    lambda row: text_similarity(row['Role'], row['Transcript']), axis=1
)

In [14]:
from textblob import TextBlob
data['cultural_fit_sentiment'] = data['Reason for decision'].apply(lambda x: TextBlob(x).sentiment.polarity)
#Job Description to Transcript Sentiment Gap
data['jd_transcript_sentiment_gap'] = data['sentiment'] - data['cultural_fit_sentiment']
#Job Description Length
data['job_desc_length'] = data['Job Description'].str.split().apply(len)
#Role to Resume Similarity
data['role_resume_similarity'] = data.apply(
    lambda row: text_similarity(row['Role'], row['Resume']), axis=1
)
#Combined Text Similarity
data['combined_text_similarity'] = (
    data['resume_jd_similarity'] + data['resume_transcript_similarity']
) / 2

In [16]:
!pip install textstat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 30.3 MB/s eta 0:00:00


In [17]:
#Sentiment to Lexical Diversity Ratio
data['sentiment_to_diversity_ratio'] = data['sentiment'] / data['lexical_diversity']
#clarity score
import textstat
data['clarity_score'] = data['Transcript'].apply(lambda x: textstat.flesch_reading_ease(x))
#confidence score
data['confidence_score'] = data['Transcript'].apply(lambda x: x.count('I think') + x.count('Maybe'))
#Clarity and Confidence Interaction
data['clarity_confidence_interaction'] = data['clarity_score'] * data['confidence_score']
#Soft Skills
from textblob import TextBlob

data['soft_skills_sentiment'] = data['Transcript'].apply(lambda x: TextBlob(x).sentiment.polarity)
# Length of transcript (number of characters)
data['transcript_length_characters'] = data['Transcript'].apply(len)

In [18]:
# Function to compute similarity score between Resume and Job Description
def compute_similarity(text1, text2):
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform([text1, text2])
    return cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]

# Calculate technical skill matching score
data['technical_skill_match'] = data.apply(lambda row: compute_similarity(row['Resume'], row['Job Description']), axis=1)
# Function to compute similarity score between Resume and Job Description
def compute_similarity(text1, text2):
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform([text1, text2])
    return cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]

# Calculate technical skill matching score
data['technical_skill_match2'] = data.apply(lambda row: compute_similarity(row['Resume'], row['Transcript']), axis=1)
# Function to compute similarity score between Resume and Job Description
def compute_similarity(text1, text2):
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform([text1, text2])
    return cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]

# Calculate technical skill matching score
data['technical_skill_match3'] = data.apply(lambda row: compute_similarity(row['Job Description'], row['Transcript']), axis=1)

In [20]:
data['job_description_experience_match'] = data.apply(lambda row: len(set(row['Resume'].split()) & set(row['Job Description'].split())), axis=1)

In [21]:
#job score
def job_fit_analysis(job_desc, transcript):
    # You can use similarity or keyword matching here
    job_keywords = job_desc.split()
    transcript_keywords = transcript.split()
    common_keywords = set(job_keywords).intersection(transcript_keywords)
    return len(common_keywords) / len(job_keywords)

data['job_fit_score'] = data.apply(lambda row: job_fit_analysis(row['Job Description'], row['Transcript']), axis=1)
#job description complexity
data['job_desc_complexity'] = data['Job Description'].apply(lambda x: textstat.flesch_reading_ease(x))
data['num_words_in_transcript'] = data['Transcript'].apply(lambda x: len(x.split()))
#interaction quality check
data['interaction_quality'] = data['num_words_in_transcript'] * data['sentiment']

In [22]:
# Text complexity (resume and transcript - using a simple metric like Flesch Reading Ease)
def text_complexity(text):
    # Implement text complexity (e.g., Flesch Reading Ease)
    # Here's a placeholder function:
    return len(text.split()) / len(set(text.split()))  # A basic metric

data['text_complexity_transcript'] = data['Transcript'].apply(text_complexity)
data['text_complexity_resume'] = data['Resume'].apply(text_complexity)
# Create a fresh TF-IDF vectorizer and fit it to the data
loaded_vectorizer = TfidfVectorizer()
loaded_vectorizer.fit(data['Job Description'].fillna(''))

TfidfVectorizer()

In [24]:
# Transform text data
new_job_desc_vectors = loaded_vectorizer.transform(data['Job Description'].fillna(''))
new_resume_vectors = loaded_vectorizer.transform(data['Resume'].fillna(''))
new_transcript_vectors = loaded_vectorizer.transform(data['Transcript'].fillna(''))
# Cosine similarities
data['resume_jod_similarity'] = [
    cosine_similarity(new_resume_vectors[i], new_job_desc_vectors[i])[0][0] for i in range(len(data))
]
data['transcript_jod_similarity'] = [
    cosine_similarity(new_transcript_vectors[i], new_job_desc_vectors[i])[0][0] for i in range(len(data))
]
import pickle

# Load the best model from the saved file
with open('/content/drive/MyDrive/ai_recruitment_dataset/xgboost_best_model.pkl', 'rb') as model_file:
    xgb_best = pickle.load(model_file)

print("XGBoost model loaded successfully.")

XGBoost model loaded successfully.


In [27]:
print(data.columns)

Index(['Unnamed: 0.1', 'Unnamed: 0', 'ID', 'Name', 'Role', 'Transcript',
       'Resume', 'Reason for decision', 'Job Description',
       'num_words_in_transcript', 'resume_jd_similarity',
       'resume_transcript_similarity', 'sentiment', 'lexical_diversity',
       'transcript_length_words', 'reason_length', 'resume_length',
       'word_count_ratio', 'role_transcript_similarity',
       'cultural_fit_sentiment', 'jd_transcript_sentiment_gap',
       'job_desc_length', 'role_resume_similarity', 'combined_text_similarity',
       'sentiment_to_diversity_ratio', 'clarity_score', 'confidence_score',
       'clarity_confidence_interaction', 'soft_skills_sentiment',
       'transcript_length_characters', 'technical_skill_match',
       'technical_skill_match2', 'technical_skill_match3',
       'job_description_experience_match', 'job_fit_score',
       'job_desc_complexity', 'interaction_quality',
       'text_complexity_transcript', 'text_complexity_resume',
       'resume_jod_similari

In [28]:
# List of features that match the model training features
features = [
    'num_words_in_transcript', 'resume_jd_similarity',
    'resume_transcript_similarity', 'sentiment', 'lexical_diversity',
    'transcript_length_words', 'reason_length', 'resume_length',
    'word_count_ratio', 'role_transcript_similarity',
    'cultural_fit_sentiment', 'jd_transcript_sentiment_gap',
    'job_desc_length', 'role_resume_similarity', 'combined_text_similarity',
    'sentiment_to_diversity_ratio', 'clarity_score', 'confidence_score',
    'clarity_confidence_interaction', 'soft_skills_sentiment',
    'transcript_length_characters', 'technical_skill_match',
    'technical_skill_match2', 'technical_skill_match3',
    'job_description_experience_match', 'job_fit_score',
    'job_desc_complexity', 'interaction_quality',
    'text_complexity_transcript', 'text_complexity_resume'
]

# Ensure you're selecting only the relevant columns
X = data[features]

# Convert data to numeric if needed
X = X.apply(pd.to_numeric, errors='coerce')

# Make predictions using the best XGBoost model
data['Selection_Status'] = xgb_best.predict(X)

# Map the result to 'Selected' or 'Not Selected'
data['Selection_Status'] = data['Selection_Status'].apply(lambda x: 'select' if x == 1 else 'reject')
# Save only specific columns (e.g., 'Selection_Status' along with other relevant ones)
data[['Name','Selection_Status']].to_excel('predicted_selection_status.xlsx', index=False)
print("Selected columns have been saved to 'predicted_selection_status.xlsx'.")

Selected columns have been saved to 'predicted_selection_status.xlsx'.


In [32]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

# Email credentials and setup
sender_email = "khenishrikant@gmail.com"  # Your email address
receiver_email = "khenishrikant@gmail.com"  # Receiver's email address
subject = "Predicted Selection Status"
body = "Please find attached the predicted selection status Excel file."

# Create the MIME message
msg = MIMEMultipart()
msg['From'] = sender_email
msg['To'] = receiver_email
msg['Subject'] = subject

# Attach the email body text
msg.attach(MIMEText(body, 'plain'))

# Attach the Excel file
file_path = 'predicted_selection_status.xlsx'
with open(file_path, 'rb') as file:
    part = MIMEBase('application', 'octet-stream')
    part.set_payload(file.read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition', f'attachment; filename={file_path}')
    msg.attach(part)

# Connect to the SMTP server (Gmail example)
try:
    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()  # Start TLS for security
        server.login(sender_email, "mvao ctgh ikue awdw")  # Login to your email account
        server.sendmail(sender_email, receiver_email, msg.as_string())  # Send email
        print("Email with attachment has been sent successfully.")
except Exception as e:
    print(f"Error: {e}")

Error: (535, b'5.7.8 Username and Password not accepted. For more information, go to\n5.7.8  https://support.google.com/mail/?p=BadCredentials d9443c01a7336-21da414daf7sm66268945ad.175 - gsmtp')
